In [ ]:
# Random Forest
# Subset by system - HCMI with CPTAC first
    # cell line and CPTAC second
# Get pairs of cancer types
# Read first cancer pair of mbatch files
# Predict on system labels for mbatch, save to mbatch x column
# Read corresponding pair of vae-encoded files
# Predict on system labels for vae, save to vae x column
    # 22 input and VAE files, 13 mbatch files
# With and without RFE
# Use matching cross folds
# MMD as control

# From fig template for rf and rfe
    # and mmd for the i-o

In [ ]:
# Run 5
r = 'r5'

##### Imports

In [ ]:
# Data handling
import pandas as pd
import glob as glob

# Pairwise cancer type prep
import itertools

# RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

# RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# MMD
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

##### MMD function

In [ ]:
def compute_mmd(bio_mrk_1, bio_mrk_2, gamma):
    K_XX = rbf_kernel(bio_mrk_1, bio_mrk_1, gamma=gamma)
    K_XY = rbf_kernel(bio_mrk_1, bio_mrk_2, gamma=gamma)
    K_YY = rbf_kernel(bio_mrk_2, bio_mrk_2, gamma=gamma)

    m = bio_mrk_1.shape[0]
    n = bio_mrk_2.shape[0]

    mmd = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1))
    mmd += (np.sum(K_YY) - np.trace(K_YY)) / (n * (n - 1))
    mmd -= 2 * np.sum(K_XY) / (m * n)

    # Ensure the MMD value is non-negative
    mmd = np.maximum(mmd, 0)

    return np.sqrt(mmd)

##### RFE function

##### Random forest function

##### Data to memory

In [ ]:
# Input-data
# phase = 'input-data'
# phase_name = 'Input data'
input_pths = sorted(glob.glob('../results/input-data/'+r+'/'+r+'-fls/*'))
print('File count: ', len(input_pths))

In [ ]:
df_n = pd.read_csv(
        input_pths[5], sep = '\t', index_col = 0)

In [ ]:
df_n.System.value_counts()

In [ ]:
for pth_n in input_pths:
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    print(pth_n)
    print(df_n.System.unique())
    print(df_n.Cancer_type.unique())
    print(len(df_n))
    print(' ')
    # break

In [ ]:
df_n

In [ ]:
# Mbatch paths for read-in
# phase = 'mbatch-out'
# phase_name = 'Mbatch corrected'
mbatch_pths = sorted(glob.glob('../results/mbatch-out/r6/r6-fls/*'))
print('File count: ', len(mbatch_pths))

In [ ]:
df_n = pd.read_csv(
        mbatch_pths[2], sep = '\t', index_col = 0)

In [ ]:
df_n

In [ ]:
for pth_n in mbatch_pths:
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    print(pth_n)
    print(df_n.System.unique())
    print(df_n.Cancer_type.unique())
    print(len(df_n))
    print(' ')

In [ ]:
# Set mbatch paths for read-in
# phase = 'vae-out'
# phase_name = 'VAE corrected'
vae_pths = sorted(glob.glob('../results/vae-out/'+r+'/'+r+'-fls/*.tsv'))
print('File count: ', len(vae_pths))

In [ ]:
for pth_n in vae_pths:
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    print(pth_n)
    print(df_n.System.unique())
    print(df_n.Cancer_type.unique())
    print(len(df_n))
    print(' ')

In [ ]:
# Set mbatch paths for read-in
# phase = 'dann-out'
# phase_name = 'DANN corrected'
dann_pths = sorted(glob.glob('../results/vae-out/'+r+'/'+r+'_dann/*.tsv'))
print('File count: ', len(dann_pths))

In [ ]:
for pth_n in dann_pths:
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    print(pth_n)
    print(df_n.System.unique())
    print(df_n.Cancer_type.unique())
    print(len(df_n))
    print(' ')

In [ ]:
tabular results to disk, four lists
focus on  results
    itertools - prot and trans, ?
    RFE, RF on both label sets?
    MMD

4 data, input

want value counts! samples by label <--- descrptv stats
    Samples
    Cancer types
annotation
use loss plot format, dstnc

In [ ]:
prot_pths_n = glob.glob('../results/input-data/r5/r5-fls/*_proteomics_*')

In [ ]:
tran_pths_n = glob.glob('../results/input-data/r5/r5-fls/*_transcriptomics_*')
len(tran_pths_n)

#### Get pairs function

In [ ]:
def get_pairs(pths_n):
    prot_list = []
    tran_list = []
    for file in pths_n:
        if 'proteomics' in file:
            prot_list.append(file)
        elif 'transcriptomics' in file:
            tran_list.append(file)
    tran_comb = list(itertools.combinations(tran_list, 2))
    prot_comb = list(itertools.combinations(prot_list, 2))
    return tran_comb, prot_comb

#### Continue

In [ ]:
pths_n

In [ ]:
for pi, pth_n in enumerate(pths_n):
    benchmark = pth_n.split('/')[-1].split('.')[0]
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    break

In [ ]:
# add cross val

#### Pairwise subsets

In [ ]:
ls ../results/mbatch-out/r5/r5-fls/

In [ ]:
# control for within-system pairs for c_typ pairwise comparisons

In [ ]:
ls ../results/mbatch-out/r5/r5-fls/*HCMI+CPTAC.tsv

In [ ]:
ls ../results/mbatch-out/r5/r5-fls/*cell-line+CPTAC.tsv

#### Main setup

In [ ]:
r

In [ ]:
systms

In [ ]:
phase

In [ ]:
phase_name

In [ ]:
pths_n # Expecting only transcriptomics for both sys cmbs
            # 2 sys combos - HCMI+CPTAC and cell_line+CPTAC
        # because reading from mbatch and only trans ran

In [ ]:
from IPython.display import Image
Image("../../F1_scatter.png")

In [ ]:
pth_cmb_n

In [ ]:
# 15 dann files for comparison

In [ ]:
ls ../results/vae-out/r5/r5_dann/*trans*.tsv

In [ ]:
# 15 vae files for comparison

In [ ]:
ls ../results/vae-out/r5/r5-fls/*trans*.tsv

#### Main

In [ ]:
vae1

In [ ]:
pths_n

In [ ]:
r = 'r5'

phase = 'mbatch-out'

systms = 'HCMI+CPTAC'
# systms = 'cell-line+CPTAC'

pths_n = sorted(
    glob.glob('../results/'+phase+
              '/'+r+'/'+r+'-fls/*'+systms+'.tsv'))

tran_comb, prot_comb = get_pairs(pths_n)

# Clf, from Set tran_comb

for cmb_n, pth_cmb_n in enumerate(tran_comb):

    mb_n1 = pd.read_csv(
        pth_cmb_n[0], sep = '\t', index_col = 0)
    print(len(mb_n1))
    mb_n2 = pd.read_csv(
        pth_cmb_n[1], sep = '\t', index_col = 0)
    print(len(mb_n1))
    mb_cmb = pd.concat([mb_n1, mb_n2], axis = 0)

    vae_pth_1 = '../results/dann-out/r5/r5-fls/' + pth_cmb_n[0].split('/')[-1]
    vae_n1 = pd.read_csv(
        vae_pth_1, sep = '\t', index_col = 0)
    print(len(vae_n1))
    vae_pth_2 = '../results/dann-out/r5/r5-fls/' + pth_cmb_n[1].split('/')[-1]
    vae_n2 = pd.read_csv(
        vae_pth_2, sep = '\t', index_col = 0)
    print(len(vae_n2))
    vae_cmb = pd.concat([vae_n1, vae_n2], axis = 0)

    Xmb = mb_cmb.iloc[:, 2:]
    ymb_sys = mb_cmb.System
    ymb_cncr = mb_cmb.Cancer_type

    Xvae = vae_cmb.iloc[:, 2:]
    yvae_sys = vae_cmb.System
    yvae_cncr = vae_cmb.Cancer_type    

    sys_scores = []
    for xf_i in list(range(1,10)):
        Xmb_train, Xmb_test, ymb_train, ymb_test = train_test_split(
                                                        Xmb, ymb_sys)
        
        break
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        sys_scores.append(score)
    break
    cncr_scores = []
    for xf_i in list(range(1,10)):
        X_train, X_test, y_train, y_test = train_test_split(
                                                    X, y_cncr)
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        cncr_scores.append(score)    
    break

In [ ]:
vae_pth_2

In [ ]:
vae_pth_1

In [ ]:
pth_cmb_n[1]

In [ ]:
pth_cmb_n[0]

In [ ]:
pwd

In [ ]:
ls ../results/input-data/r5/r5-fls/

In [ ]:
# Relabel HCMI files
r = 'r5'
phase = 'mbatch-out'
systms = 'HCMI+CPTAC'
pths_n = sorted(
    glob.glob('../results/'+phase+
              '/'+r+'/'+r+'-fls/*'+systms+'.tsv'))
for pth in pths_n:
    print(pth)
    df = pd.read_csv(
        pth, sep = '\t', index_col = 0)
    df['System'] = df['System'].replace('cell_line', 'HCMI')
    df.to_csv(
        pth,
        sep = '\t')
    # break

In [ ]:
# x is system, need two values: mbatch and vae
# y is cancer type, need two values: mbatch and vae

# start with proteomics <-- subset scatter plots level one - No, only trans in Mbatch

    # make an mbatch prediction

        # within HCMI+CPTAC <-- subset scatter plot level two,
                                            # this means two scatters
                                            # to Follow-ups
        # for brain <-- start
        # on system label <-- this is the x value in the scatter
        # ^ this is one of six dot pairs for scatter plot one

    # make vae corresponding ^ prediction

In [ ]:
df_n.Cancer_type.value_counts() # This will be more meaningful
                                # for pairwise

In [ ]:
df_n.System.value_counts()